## Вопросы для самопроверки:
* что такое обучени с подкреплением (reinforcement learning)?
* что такое среда?
* что такое агент?
* что такое награда, какая она может быть?

#  FrozenLake
Today you are going to learn how to survive walking over the (virtual) frozen lake through discrete optimization.

<img src="http://vignette2.wikia.nocookie.net/riseoftheguardians/images/4/4c/Jack's_little_sister_on_the_ice.jpg/revision/latest?cb=20141218030206" alt="a random image to attract attention" style="width: 400px;"/>


In [46]:
import gym
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt
from collections import defaultdict


#create a single game instance
env = gym.make("Taxi-v2")

#start new game
env.reset()

[2017-12-25 01:11:48,180] Making new env: Taxi-v2


In [14]:
# display the game state
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



### legend

![img](https://cdn-images-1.medium.com/max/800/1*MCjDzR-wfMMkS0rPqXSmKw.png)

### Gym interface

The three main methods of an environment are
* __reset()__ - reset environment to initial state, _return first observation_
* __render()__ - show current environment state (a more colorful version :) )
* __step(a)__ - commit action __a__ and return (new observation, reward, is done, info)
 * _new observation_ - an observation right after commiting the action __a__
 * _reward_ - a number representing your reward for commiting action __a__
 * _is done_ - True if the MDP has just finished, False if still in progress
 * _info_ - some auxilary stuff about what just happened. Ignore it for now

In [15]:
print("initial observation code:", env.reset())
print('printing observation:')
env.render()
print("observations:", env.observation_space, 'n=', env.observation_space.n)
print("actions:", env.action_space, 'n=', env.action_space.n)

initial observation code: 308
printing observation:
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

observations: Discrete(500) n= 500
actions: Discrete(6) n= 6


In [16]:
print("taking action 2 (right)")
new_obs, reward, is_done, _ = env.step(2)
print("new observation code:", new_obs)
print("reward:", reward)
print("is game over?:", is_done)
print("printing new state:")
env.render()

taking action 2 (right)
new observation code: 308
reward: -1
is game over?: False
printing new state:
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)


In [17]:
action_to_i = {
    'right': 2,
    'up': 1,
    'down': 0,
    'left': 3,
    'pickup': 4,
    'dropoff': 5,
}

### Play with it
* Try walking 5 steps without falling to the (H)ole
 * Bonus quest - get to the (G)oal
* Sometimes your actions will not be executed properly due to slipping over ice
* If you fall, call __env.reset()__ to restart

In [18]:
np.random.seed(123)
env.seed(123)

[123]

In [24]:
env.reset()
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [25]:
env.step(action_to_i['left'])
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)


In [27]:
env.step(action_to_i['up'])
env.step(action_to_i['up'])
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)


In [28]:
env.step(action_to_i['right'])
env.step(action_to_i['up'])
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)


### Baseline: random search

### Policy

* The environment has a 4x4 grid of states (16 total), they are indexed from 0 to 15
* From each states there are 4 actions (left,down,right,up), indexed from 0 to 3

We need to define agent's policy of picking actions given states. Since we have only 16 disttinct states and 4 actions, we can just store the action for each state in an array.

This basically means that any array of 16 integers from 0 to 3 makes a policy.

In [29]:
n_states = env.observation_space.n
n_actions = env.action_space.n

def get_random_policy():
    """
    Build a numpy array representing agent policy.
    This array must have one element per each of 16 environment states.
    Element must be an integer from 0 to 3, representing action
    to take from that state.
    """
    return np.random.choice(range(n_actions), size=n_states)

In [30]:
np.random.seed(1234)
policies = [get_random_policy() for i in range(10**4)]
assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should match n_actions-1'
action_probas = np.unique(policies, return_counts=True)[-1] /10**4. /n_states
print("Action frequencies over 10^4 samples:",action_probas)
assert np.allclose(action_probas, [1. / n_actions] * n_actions, atol=0.05), "The policies aren't uniformly random (maybe it's just an extremely bad luck)"
print("Seems fine!")

Action frequencies over 10^4 samples: [ 0.1668264  0.1667818  0.166578   0.166587   0.166508   0.1667188]
Seems fine!


### Let's evaluate!
* Implement a simple function that runs one game and returns the total reward

In [31]:
def sample_reward(env, policy, t_max=100):
    """
    Interact with an environment, return sum of all rewards.
    If game doesn't end on t_max (e.g. agent walks into a wall), 
    force end the game and return whatever reward you got so far.
    Tip: see signature of env.step(...) method above.
    """
    s = env.reset()
    total_reward = 0
    
    for _ in range(t_max)
        s, r, done, _ = env.step(policy[s])
        total_reward += r
        if done:
            break
    return total_reward

In [32]:
print("generating 10^3 sessions...")
rewards = [sample_reward(env, get_random_policy()) for _ in range(1000)]
assert all([type(r) in (int, float) for r in rewards]), 'sample_reward must return a single number'

print("Looks good!")

generating 10^3 sessions...
Looks good!


In [33]:
def evaluate(policy, n_times=100):
    """Run several evaluations and average the score the policy gets."""
    rewards = [sample_reward(env, policy) for i in range(n_times)]
    return float(np.mean(rewards))
        

### Main loop

In [13]:
best_policy = None
best_score = -float('inf')

from tqdm import tqdm
for i in tqdm(range(10000)):
    policy = get_random_policy()
    score = evaluate(policy)
    if score > best_score:
        best_score = score
        best_policy = policy
        print("New best score:", score)


  0%|          | 1/10000 [00:00<29:04,  5.73it/s]

New best score: -592.12


  0%|          | 6/10000 [00:00<23:47,  7.00it/s]

New best score: -511.39


  0%|          | 18/10000 [00:02<19:11,  8.67it/s]

...

 74%|███████▍  | 7400/10000 [09:22<03:17, 13.17it/s]

New best score: -405.01


 81%|████████  | 8096/10000 [10:14<02:24, 13.17it/s]

New best score: -359.92


100%|██████████| 10000/10000 [12:39<00:00, 13.17it/s]


# Part II Genetic algorithm 

The next task is to devise some more effecient way to perform policy search.
We'll do that with a bare-bones evolutionary algorithm.
[unless you're feeling masochistic and wish to do something entirely different which is bonus points if it works]

In [37]:
def crossover(policy1, policy2, p=0.5):
    """
    for each state, with probability p take action from policy1, else policy2
    """
    n = len(policy1)
    first = np.random.rand(n) < p
    return policy1 * first + policy2 * (1 - first)

In [38]:
def mutation(policy, p=0.1):
    """
    for each state, with probability p replace action with random action
    Tip: mutation can be written as crossover with random policy
    """
    return crossover(get_random_policy(), policy, p)
    

In [39]:
np.random.seed(1234)
policies = [crossover(get_random_policy(), get_random_policy()) for i in range(10**4)]

assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should be n_actions-1'

assert any([np.mean(crossover(np.zeros(n_states), np.ones(n_states))) not in (0, 1)
               for _ in range(100)]), "Make sure your crossover changes each action independently"
print("Seems fine!")

Seems fine!


In [40]:

n_epochs = 100 #how many cycles to make
pool_size = 100 #how many policies to maintain
n_crossovers = 50 #how many crossovers to make on each step
n_mutations = 50 #how many mutations to make on each tick


In [41]:
print("initializing...")
pool = [get_random_policy() for _ in range(pool_size)]
pool_scores = [evaluate(policy) for policy in pool]


initializing...


In [19]:
assert type(pool) == type(pool_scores) == list
assert len(pool) == len(pool_scores) == pool_size
assert all([type(score) in (float, int) for score in pool_scores])


In [42]:
#main loop
for epoch in range(n_epochs):    
    crossovered = [crossover(pool[np.random.randint(pool_size)],
                             pool[np.random.randint(pool_size)])
                   for _ in range(n_crossovers)]
    mutated = [mutation(pool[np.random.randint(pool_size)])
               for _ in range(n_mutations)]
    
    assert type(crossovered) == type(mutated) == list
    
    #add new policies to the pool
    pool.extend(crossovered)
    pool.extend(mutated)
    
    pool_scores = [evaluate(policy) for policy in pool]
    
    #select pool_size best policies
    selected_indices = np.argsort(pool_scores)[-pool_size:]
    pool = [pool[i] for i in selected_indices]
    pool_scores = [pool_scores[i] for i in selected_indices]

    #print the best policy so far (last in ascending score order)
    if (epoch + 1) % 10 == 0:
        print("Epoch", epoch)
        print("best score:", pool_scores[-1])

Epoch 0:
best score: -413.47
Epoch 1:
best score: -440.02
Epoch 2:
best score: -440.29
Epoch 3:
best score: -405.1
Epoch 4:
best score: -413.38
Epoch 5:
best score: -393.94
Epoch 6:
best score: -350.83
Epoch 7:
best score: -323.65
Epoch 8:
best score: -359.56
Epoch 9:
best score: -342.1
Epoch 10:
best score: -306.01
Epoch 11:
best score: -341.2
Epoch 12:
best score: -324.19
Epoch 13:
best score: -341.02
Epoch 14:
best score: -323.83
Epoch 15:
best score: -260.83
Epoch 16:
best score: -287.92
Epoch 17:
best score: -279.37
Epoch 18:
best score: -296.92
Epoch 19:
best score: -278.74
Epoch 20:
best score: -278.11
Epoch 21:
best score: -288.1
Epoch 22:
best score: -279.55
Epoch 23:
best score: -279.55
Epoch 24:
best score: -252.28
Epoch 25:
best score: -261.46
Epoch 26:
best score: -243.46
Epoch 27:
best score: -261.37
Epoch 28:
best score: -260.83
Epoch 29:
best score: -261.28
Epoch 30:
best score: -252.28
Epoch 31:
best score: -225.64
Epoch 32:
best score: -207.46
Epoch 33:
best score: -2

Ссылка на фидбек по семинару: [link](https://docs.google.com/forms/d/e/1FAIpQLSf-08wFrEke6zKlysETYiqAjH5CRXtOKut5Q77Tr5rdVId7zA/)

# QLearning

In [50]:
class Agent():
    def __init__(self, learning_rate, epsilon, discount, n_actions):
        self.actions= range(n_actions)
        self.qvalues = defaultdict(lambda : defaultdict(float))
        self.learning_rate = learning_rate
        self.epsilon = epsilon
        self.discount = discount

    def act(self, state):
        if np.random.rand() < self.epsilon:
            return np.random.choice(self.actions)
        return self.actions[np.argmax([self.qvalues[state][action] for action in self.actions])]

    def update(self, state, action, next_state, reward):
        value = np.max([self.qvalues[next_state][action] for action in self.actions])
        reference_qvalue = self.discount * value + reward
        updated_qvalue = (1 - self.learning_rate) * self.qvalues[state][action] + self.learning_rate * reference_qvalue
        self.qvalues[state][action] = updated_qvalue


In [52]:
agent = Agent(discount=0.99, learning_rate=0.3, epsilon=0.1, n_actions=6)

rewards = []

for i in range(10**4):
    s = env.reset()
    total_reward = 0
    
    for _ in range(10**4):
        a = agent.act(s)
        new_s, r, done, _ = env.step(a)
        agent.update(s, a, new_s, r)
        s = new_s
        total_reward += r
        if done:
            break
    
    rewards.append(total_reward)
    agent.epsilon *= 0.995

    if (i + 1) % 100 ==0:
        print("{}. Mean reward over last 300 steps: {}\r".format(i, np.mean(rewards[-300:])))
    if (i + 1) % 1000 ==0:
        print("\n")


99. Mean reward over last 300 steps: -258.49
199. Mean reward over last 300 steps: -183.185
299. Mean reward over last 300 steps: -130.34333333333333
399. Mean reward over last 300 steps: -47.22666666666667
499. Mean reward over last 300 steps: -10.26
599. Mean reward over last 300 steps: -0.77
699. Mean reward over last 300 steps: 4.366666666666666
799. Mean reward over last 300 steps: 5.693333333333333
899. Mean reward over last 300 steps: 6.863333333333333
999. Mean reward over last 300 steps: 7.43


1099. Mean reward over last 300 steps: 7.886666666666667
1199. Mean reward over last 300 steps: 8.09
1299. Mean reward over last 300 steps: 8.13
1399. Mean reward over last 300 steps: 7.993333333333333
1499. Mean reward over last 300 steps: 8.206666666666667
1599. Mean reward over last 300 steps: 8.323333333333334
1699. Mean reward over last 300 steps: 8.453333333333333
1799. Mean reward over last 300 steps: 8.413333333333334
1899. Mean reward over last 300 steps: 8.306666666666667
1999